In [1]:
import json
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
import rasterio as rio
from rasterio.plot import show
from rasterio.mask import mask
from shapely.geometry import Polygon

In [2]:
class Box:
    def __init__(self, left:int=0, bottom:int=0, right:int=0, top:int=0):
        self.left,self.bottom,self.right,self.top = left,bottom,right,top
        self.width, self.height = self.right -self.left, self.top -self.bottom
    def __str__(self):
        return "left:{} bottom:{} right:{} top:{} width:{} height:{}".format(
            self.left, self.bottom, self.right, self.top, self.width, self.height)
    def in_bounds(self, x:int, y:int) -> bool:
        return self.left < x < self.right and self.bottom < y < self.top

In [3]:
def center(poly:Polygon):
    return(int(poly.bounds[0]+poly.bounds[2]-poly.bounds[0]),
           int(poly.bounds[1]+poly.bounds[3]-poly.bounds[1]))

In [4]:
def get_shapes_in_box(gdf: GeoDataFrame, box: Box) -> GeoDataFrame:
    return gdf[gdf.geometry.apply(
        lambda d: box.in_bounds(
            x = center(Polygon(d))[0],
            y = center(Polygon(d))[1]
        ))]

In [5]:
def shape_mean(tif, shape):
    out_image, out_transform = mask(tif, shapes=[shape], crop=True, invert=False)
    li = []
    for y in out_image[0]:
        for x in y:
            if x > 0:
                li.append(x)
    return np.array(li).mean()

In [6]:
shapefile = gpd.read_file("/home/becode/Projects/3D_House/Shapes/ANTWERPEN_L72_2020/Bpn_ReBu.shp")

In [7]:
beacon_tif = rio.open("./k15_1_2_1_1_0.tif")
beacon_box = Box(*beacon_tif.bounds)

In [8]:
beacon_shapes = get_shapes_in_box(shapefile, beacon_box)
beacon_shapes.reset_index(inplace=True)
#beacon_shapes = beacon_shapes.drop(
#    ["index","RecId","CaBlKey","AdMuKey","Type","Source","Quality","QualDate","UpdDate"], axis=1)
beacon_shapes["Height"] = beacon_shapes.geometry.apply(lambda x: shape_mean(beacon_tif, x))
#beacon_shapes

/home/becode/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: Mean of empty slice.
  
/home/becode/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:831: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [9]:
minx = beacon_shapes.bounds.minx.min()
miny = beacon_shapes.bounds.miny.min()
maxx = beacon_shapes.bounds.maxx.max()
maxy = beacon_shapes.bounds.maxy.max()
print(" , ".join(map(str,[minx, miny, maxx, maxy])))

151974.0570291095 , 212336.08812237438 , 152999.74696514395 , 212999.84792066645


# Downscale the polygons

In [10]:
def downscale(poly:Polygon, minx:float, miny:float) -> Polygon:
    li = []
    for coord in poly.exterior.coords:
        coord = ((coord[0]-minx)/100, (coord[1]-miny)/100)
        li.append(coord)
    return Polygon(li)

beacon_shapes["Downscale"] = beacon_shapes.geometry.apply(
    lambda x: downscale(x,minx,miny))
beacon_shapes["Height"] = beacon_shapes.Height.apply(
    lambda x: x/100)

/home/becode/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:831: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [11]:
polygons = pd.DataFrame([beacon_shapes.Height, beacon_shapes.Downscale]).T
polygons.to_csv("./downscaled_polygons.csv")

In [12]:
#polygons

# EXPERIMENT

In [13]:
from blender_object_builder import Vertex

In [14]:
def polypoint_to_vertex(polypoint):
    return Vertex(*polypoint)
def polygon_to_vertices(polygon):
    li = []
    for coord in polygon.exterior.coords:
        li.append(polypoint_to_vertex(coord))
    return li[:-1]

beacon_shapes["Vertices"] = beacon_shapes.Downscale.apply(
    lambda x: polygon_to_vertices(x))

vertices = pd.DataFrame(beacon_shapes[["Vertices", "Height"]])
vertices.to_csv("./experimental.csv")

/home/becode/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:831: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


# JSON dump

### Experiment 1

In [15]:
#data = {}
#for idx, v in enumerate(vertices.Vertices):
#    data[idx] = []
#    for vertex in v:
#        data[idx].append(tuple(vertex.xyz))
#with open("experimental.json", "w") as out:
#    json.dump(data, out)

### Experiment 2

In [16]:
data = {}
for idx, entry in vertices.iterrows():
    data[idx] = {}
    data[idx]["Height"] = entry.Height
    data[idx]["Vertices"] = []
    for vertex in entry.Vertices:
        data[idx]["Vertices"].append(tuple(vertex.xyz))
with open("experimental.json", "w") as out:
    json.dump(data, out)

### Experiment 3 RAW height map

In [3]:
# JSON File
raw_tif = rio.open("./crop.tif")
raw_data = np.array(raw_tif.read(1))
data = {}
for y in range(raw_data.shape[0]):
    data[int(y)] = []
    for x in range(raw_data.shape[1]):
        data[y].append(np.float64(raw_data[y,x]))
with open("raw_experiment.json", "w") as out:
    json.dump(data, out)

In [11]:
# BYTE ARRAY File
raw = np.array(rio.open("./k15_1_2_1_1_0.tif").read(1))
with open("K15_1_2_1_1_0.dada", "wb") as out:
    for y in range(raw.shape[0]):
        for x in range(raw.shape[1]):
            b = bytearray()
            b.extend(raw[y,x])
            out.write(b)

##### Conclusion: .tif uses a byte array as "filetype" so impossible to go smaller without compression